<a href="https://colab.research.google.com/github/DRIP-AI-RESEARCH-JUNIOR/MUSIC_GENEARATION/blob/master/MusicGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class Generator(nn.Module):
    def __init__(self,num_features,p=0.1):
        super(Generator, self).__init__()
        self.fc_encoder = nn.Linear(2*num_features,num_features)
        self.lstmCell_encoder = nn.LSTMCell(input_size=num_features, hidden_size=num_features)
        self.dropout = nn.Dropout(p=p)
        self.fc_decoder = nn.Linear(num_features,num_features)
        self.lstmCell_decoder = nn.LSTMCell(input_size=num_features, hidden_size=num_features)

    def weight_init(self):
        torch.nn.init.zeros_(tensor)

    def forward(self,x,he,ce,hd,cd):
        output = []
        seq_len = x.size()[1]
        batch = x.size()[0]
        num_features = x.size()[2]
        input = x.permute(1,0,2)
        start = nn.init.uniform_(torch.empty(batch,num_features))
        for x_step in input:
            input_concat = torch.cat((x_step,start),dim=-1)
            linear_out_encoder = F.relu(self.fc_encoder(input_concat))
            he,ce = self.lstmCell_encoder(linear_out_encoder,(he,ce))
            he = self.dropout(he)
            hd,cd = self.lstmCell_decoder(he,(hd,cd))
            start = F.sigmoid(self.fc_decoder(hd))
            output.append(start)
        output = torch.stack(output)
        output = output.permute(1,0,2)
        return output


In [18]:
x = torch.randn(2,3,88)
he = torch.randn(2,88)
ce = torch.randn(2,88)
hd = torch.randn(2,88)
cd = torch.randn(2,88)

In [33]:
model = Generator(88)
out = model(x,he,ce,hd,cd)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [34]:
print(out.shape)

torch.Size([2, 3, 88])


In [39]:
class Discriminator(nn.Module):
    def __init__(self,num_feature,p=0.1):
        super(Discriminator,self).__init__()
        self.dropout = nn.Dropout(p=p)
        self.lstm = nn.LSTM(num_feature,int(num_feature/2),num_layers=2,batch_first=True,bidirectional=True)
        self.fc = nn.Linear(num_feature,1)

    def forward(self,x,h,c):
        drop_D = self.dropout(x)
        out,(h,c) = self.lstm(x,(h,c))
        out = F.sigmoid(self.fc(out))
        return out

In [41]:
x = torch.randn(2,3,88)
h = torch.randn(4,2,44)
c = torch.randn(4,2,44)
model = Discriminator(88)
out = model(x,h,c)
print(out.shape)

torch.Size([2, 3, 1])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
